#LSTM Two types : 

*   Single directional
*   Bi-Directional

**N.B : Single directional LSTM is described here**



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 1.4MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
import gensim
print(gensim.__version__)

3.8.3


#Model selection catagory - wise
<pre>
1.   <b>Word2vec model :</b>
      -> 'word2vec-google-news-300'
2.   <b>Glove model :</b> 
     *  Twitter Glove Embeddings
        -> 'glove-twitter-25'
        -> 'glove-twitter-50'
        -> 'glove-twitter-100'
        -> 'glove-twitter-200'
     *  Wikipedia Glove Embeddings
        -> 'glove-wiki-gigaword-50'
        -> 'glove-wiki-gigaword-100'
        -> 'glove-wiki-gigaword-200'
        -> 'glove-wiki-gigaword-300'
3.   <b>FastText model :</b>
        -> 'fasttext-wiki-news-subwords-300'
</pre>

In [4]:
import gensim.downloader as api
word2vec_model = api.load('fasttext-wiki-news-subwords-300')

[=================================================-] 99.5% 954.0/958.4MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
import json
import random
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.metrics import TruePositives,FalsePositives,TrueNegatives,FalseNegatives,Precision,Recall,Accuracy,confusion_matrix

In [6]:
def find_best(training_details):
  loss = []
  accuracy = []
  val_loss = []
  val_accuracy = []

  for i in range(len(training_details)):
    loss.append(training_details[i]['loss'])
    accuracy.append(training_details[i]['accuracy'])
    val_loss.append(training_details[i]['val_loss'])
    val_accuracy.append(training_details[i]['val_accuracy'])
  
  max_accuracy = max(accuracy)
  max_accuracy_indices = [i for i, x in enumerate(accuracy) if x == max_accuracy]

  temp_val_accuracy = []
  if len(max_accuracy_indices)>1:
    for i in max_accuracy_indices:
      temp_val_accuracy.append(val_accuracy[i])
    
    max_val_accuracy = max(temp_val_accuracy)

    max_val_accuracy_indices = [i for i, x in enumerate(val_accuracy) if x == max_val_accuracy]
    
    temp_loss = []
    if len(max_val_accuracy_indices)>1:
      for i in max_val_accuracy_indices:
        temp_loss.append(loss[i])
      
      min_loss = min(temp_loss)
      
      min_loss_indices = [i for i, x in enumerate(loss) if x == min_loss]

      temp_val_loss = []

      if len(min_loss_indices)>1:
        for i in min_loss_indices:
          temp_val_loss.append(val_loss[i])
        
        min_val_loss = min(temp_val_loss)

        min_val_loss_indices = [i for i, x in enumerate(val_loss) if x == min_val_loss]

        if len(min_val_loss_indices)>1:
          return random.choice(min_val_loss_indices)
        else:
          return min_val_loss_indices
      else:
        return min_loss_indices
    else:
      return max_val_accuracy_indices
  else:
    return max_accuracy_indices

In [7]:
f = open('gdrive/My Drive/Colab Notebooks/Soft_computing/SoftComputing_project/final_data.json') 
data = json.load(f)
f1 = open('gdrive/My Drive/Colab Notebooks/Soft_computing/SoftComputing_project/class_value.json') 
label = json.load(f1)
# label = np.array(label)

In [8]:
train_and_valid_data, test_data, train_and_valid_label, test_label = train_test_split(data,label, test_size=0.2, random_state=1)
 #now , train and validation data = 0.8 portion data of total 1.0 portion data , now we divide this 0.8 portion data to train and valid data 

In [9]:
train_data,valid_data,train_label,valid_label = train_test_split(train_and_valid_data,train_and_valid_label, test_size=0.25, random_state=1)
# as main data is 0.8 so here train data = 0.8*0.75 = 0.6  and valid data = 0.8* 0.25 = 0.2

In [10]:
print(len(train_data))
print(len(test_data))
print(len(valid_data))

36541
12181
12181


In [11]:
train_label = np.array(train_label)
valid_label = np.array(valid_label)
test_label = np.array(test_label)

In [12]:
vocab_size = 30000
embedding_dim_normal_embedding = 64
max_length = 32
# trunc_type='post'
oov_tok = "<OOV>"

# tokenizer = Tokenizer(num_words = vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(train_data)
training_padded = pad_sequences(training_sequences,maxlen=max_length) #truncating=trunc_type

valid_sequences = tokenizer.texts_to_sequences(valid_data)
valid_padded = pad_sequences(valid_sequences,maxlen=max_length) #truncating=trunc_type

testing_sequences = tokenizer.texts_to_sequences(test_data)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)# truncating=trunc_type

In [13]:
total_words = len(word_index)
print(total_words)

23596


In [14]:
embedding_dim_pretrained_embedding = word2vec_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim_pretrained_embedding))
for word, i in word_index.items():
  try:
    embedding_matrix[i] = word2vec_model[word]
  except:
    pass

In [15]:
# print(total_words)

In [16]:
# len(embedding_matrix[0])

In [17]:
tf.keras.backend.clear_session()

#Tune Batch size and epoch

In [ ]:
def param_tune(epoch_num,batch_size_num):
  batch = []
  epoch = []
  training_details = []
  for i in range(len(batch_size_num)):
    # tf.keras.backend.clear_session()
    model3 = tf.keras.Sequential([
                                  tf.keras.layers.Embedding(vocab_size,embedding_dim_pretrained_embedding,weights=[embedding_matrix],input_length=max_length,trainable=False),  
                                  tf.keras.layers.LSTM(embedding_dim_pretrained_embedding),
                                  tf.keras.layers.Dropout(0.1),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                  ])
    model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    # model3.summary()
    print("Epoch : ",epoch_num,"\tBatch_size : ",batch_size_num[i])
    history = model3.fit(training_padded, train_label, epochs=epoch_num, validation_data=(valid_padded, valid_label),batch_size=batch_size_num[i])
    batch.append(batch_size_num[i]) 
    epoch.append(epoch_num)
    training_details.append({'loss': history.history['loss'][-1],
                             'accuracy':history.history['accuracy'][-1],
                             'val_loss':history.history['val_loss'][-1],
                             'val_accuracy':history.history['val_accuracy'][-1]})
    # del model3
  return batch,epoch,training_details

In [ ]:
e = 10
b_size = [16,32,64,128]
batch,epoch,training_details = param_tune(e,b_size)
for i in range(len(batch)):
  print("\n","Batch : ",batch[i]," Epoch : ",epoch[i]," Details : ",training_details[i])

571/571 [==============================] - 13s 23ms/step - loss: 0.1226 - accuracy: 0.9554 - val_loss: 0.1736 - val_accuracy: 0.9438
Epoch :  10 	Batch_size :  128
Epoch 1/10
286/286 [==============================] - 9s 33ms/step - loss: 0.2268 - accuracy: 0.9172 - val_loss: 0.1808 - val_accuracy: 0.9389
Epoch 2/10
286/286 [==============================] - 9s 31ms/step - loss: 0.1621 - accuracy: 0.9460 - val_loss: 0.1679 - val_accuracy: 0.9438
Epoch 3/10
286/286 [==============================] - 9s 31ms/step - loss: 0.1526 - accuracy: 0.9487 - val_loss: 0.1622 - val_accuracy: 0.9445
Epoch 4/10
286/286 [==============================] - 9s 31ms/step - loss: 0.1511 - accuracy: 0.9493 - val_loss: 0.1608 - val_accuracy: 0.9452
Epoch 5/10
286/286 [==============================] - 9s 31ms/step - loss: 0.1390 - accuracy: 0.9523 - val_loss: 0.1576 - val_accuracy: 0.9457
Epoch 6/10
286/286 [==============================] - 9s 31ms/step - loss: 0.1305 - accuracy: 0.9545 - val_loss: 0.1604 -

In [ ]:
indices = find_best(training_details)
print("Best parameter is : ","Batch : ",batch[indices[0]])

Best parameter is :  Batch :  16


#Tune optimizer

In [ ]:
def param_tune(epoch_num,batch_size_num,optimizer_type):
  optimizers = []
  training_details = []
  for i in range(len(optimizer_type)):
    # tf.keras.backend.clear_session()
    model3 = tf.keras.Sequential([
                                  tf.keras.layers.Embedding(vocab_size,embedding_dim_pretrained_embedding,weights=[embedding_matrix],input_length=max_length,trainable=False),  
                                  tf.keras.layers.LSTM(embedding_dim_pretrained_embedding),
                                  # tf.keras.layers.Dropout(0.5),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                  ])
    model3.compile(loss='binary_crossentropy',optimizer=optimizer_type[i],metrics=['accuracy'])
    # model3.summary()
    print("Optimizer : ",optimizer_type[i])
    history = model3.fit(training_padded, train_label, epochs=epoch_num, validation_data=(valid_padded, valid_label),batch_size=batch_size_num)
    optimizers.append(optimizer_type[i]) 
    training_details.append({'loss': history.history['loss'][-1],
                             'accuracy':history.history['accuracy'][-1],
                             'val_loss':history.history['val_loss'][-1],
                             'val_accuracy':history.history['val_accuracy'][-1]})
    # del model3
  return optimizers,training_details

In [ ]:
e = 10
b_size = 32
opt = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam', 'Ftrl'] 
optimizers,training_details = param_tune(e,b_size,opt)
for i in range(len(optimizers)):
  print("\n","Optimizer :",optimizers[i]," Details : ",training_details[i])

2284/2284 [==============================] - 39s 17ms/step - loss: 0.0431 - accuracy: 0.9844 - val_loss: 0.2569 - val_accuracy: 0.9402
Optimizer :  Ftrl
Epoch 1/10
2284/2284 [==============================] - 36s 16ms/step - loss: 0.6759 - accuracy: 0.8088 - val_loss: 0.6674 - val_accuracy: 0.8058
Epoch 2/10
2284/2284 [==============================] - 36s 16ms/step - loss: 0.6616 - accuracy: 0.8091 - val_loss: 0.6571 - val_accuracy: 0.8058
Epoch 3/10
2284/2284 [==============================] - 36s 16ms/step - loss: 0.6523 - accuracy: 0.8091 - val_loss: 0.6484 - val_accuracy: 0.8058
Epoch 4/10
2284/2284 [==============================] - 37s 16ms/step - loss: 0.6417 - accuracy: 0.8091 - val_loss: 0.6341 - val_accuracy: 0.8058
Epoch 5/10
2284/2284 [==============================] - 36s 16ms/step - loss: 0.6169 - accuracy: 0.8091 - val_loss: 0.5963 - val_accuracy: 0.8058
Epoch 6/10
2284/2284 [==============================] - 36s 16ms/step - loss: 0.5629 - accuracy: 0.8091 - val_loss: 0

In [ ]:
indices = find_best(training_details)
print("Best parameter is : ","Optimizer :",optimizers[indices[0]])

Best parameter is :  Optimizer : Nadam


#Tune Dropout

In [ ]:
def param_tune(epoch_num,batch_size_num,optimizer_type,dropout_num):
  dropouts = []
  training_details = []
  for i in range(len(dropout_num)):
    # tf.keras.backend.clear_session()
    model3 = tf.keras.Sequential([
                                  tf.keras.layers.Embedding(vocab_size,embedding_dim_pretrained_embedding,weights=[embedding_matrix],input_length=max_length,trainable=False),  
                                  tf.keras.layers.LSTM(embedding_dim,dropout=dropout_num[i]),
                                  # tf.keras.layers.Dropout(0.5),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                  ])
    model3.compile(loss='binary_crossentropy',optimizer=optimizer_type,metrics=['accuracy'])
    # model3.summary()
    print("Dropout : ",dropout_num[i])
    history = model3.fit(training_padded, train_label, epochs=epoch_num, validation_data=(valid_padded, valid_label),batch_size=batch_size_num)
    dropouts.append(dropout_num[i]) 
    training_details.append({'loss': history.history['loss'][-1],
                             'accuracy':history.history['accuracy'][-1],
                             'val_loss':history.history['val_loss'][-1],
                             'val_accuracy':history.history['val_accuracy'][-1]})
    # del model3
  return dropouts,training_details

In [ ]:
e = 10
b_size = 16
opt = 'Nadam'
drop = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
dropouts,training_details = param_tune(e,b_size,opt,drop)
for i in range(len(dropouts)):
  print("\n","Dropout :",dropouts[i]," Details : ",training_details[i])

2284/2284 [==============================] - 40s 17ms/step - loss: 0.1177 - accuracy: 0.9587 - val_loss: 0.1648 - val_accuracy: 0.9492
Epoch 10/10
2284/2284 [==============================] - 40s 17ms/step - loss: 0.1114 - accuracy: 0.9596 - val_loss: 0.1721 - val_accuracy: 0.9492
Dropout :  0.6
Epoch 1/10
2284/2284 [==============================] - 40s 17ms/step - loss: 0.2095 - accuracy: 0.9265 - val_loss: 0.1735 - val_accuracy: 0.9417
Epoch 2/10
2284/2284 [==============================] - 40s 17ms/step - loss: 0.1736 - accuracy: 0.9409 - val_loss: 0.1758 - val_accuracy: 0.9423
Epoch 3/10
2284/2284 [==============================] - 40s 17ms/step - loss: 0.1601 - accuracy: 0.9457 - val_loss: 0.1578 - val_accuracy: 0.9452
Epoch 4/10
2284/2284 [==============================] - 40s 17ms/step - loss: 0.1549 - accuracy: 0.9474 - val_loss: 0.1639 - val_accuracy: 0.9444
Epoch 5/10
2284/2284 [==============================] - 40s 17ms/step - loss: 0.1494 - accuracy: 0.9485 - val_loss: 0.1

In [ ]:
indices = find_best(training_details)
print("Best parameter is : ","Dropout :",dropouts[indices[0]])

Best parameter is :  Dropout : 0.1


#Tune hidden node

In [ ]:
def param_tune(epoch_num,batch_size_num,optimizer_type,dropout_num,neuron_num):
  neurons_num = []
  training_details = []
  for i in range(len(neuron_num)):
    # tf.keras.backend.clear_session()
    model3 = tf.keras.Sequential([
                                  tf.keras.layers.Embedding(vocab_size,embedding_dim_pretrained_embedding,weights=[embedding_matrix],input_length=max_length,trainable=False), 
                                  tf.compat.v1.keras.layers.CuDNNLSTM(neuron_num[i]),
                                  tf.keras.layers.Dropout(dropout_num),
                                  tf.keras.layers.Dense(1,activation='sigmoid')
                                  ])
    model3.compile(loss='binary_crossentropy',optimizer=optimizer_type,metrics=['accuracy'])
    # model3.summary()
    print("Neuron : ",neuron_num[i])
    history = model3.fit(training_padded, train_label, epochs=epoch_num, validation_data=(testing_padded, test_label),batch_size=batch_size_num)
    neurons_num.append(neuron_num[i]) 
    training_details.append({'loss': history.history['loss'][-1],
                             'accuracy':history.history['accuracy'][-1],
                             'val_loss':history.history['val_loss'][-1],
                             'val_accuracy':history.history['val_accuracy'][-1]})
    # del model3
  return neurons_num,training_details

In [ ]:
e = 10
b_size = 16
opt = 'Adam'
drop = 0.1
neu_ron = [embedding_dim,256,128,64,32,16]
neurons_num,training_details = param_tune(e,b_size,opt,drop,neu_ron)
for i in range(len(neurons_num)):
  print("\n","Neuron :",neurons_num[i]," Details : ",training_details[i])

Neuron :  300
Epoch 1/10
2284/2284 [==============================] - 58s 25ms/step - loss: 0.1733 - accuracy: 0.9430 - val_loss: 0.1560 - val_accuracy: 0.9489
Epoch 2/10
2284/2284 [==============================] - 54s 24ms/step - loss: 0.1465 - accuracy: 0.9498 - val_loss: 0.1534 - val_accuracy: 0.9488
Epoch 3/10
2284/2284 [==============================] - 54s 24ms/step - loss: 0.1357 - accuracy: 0.9532 - val_loss: 0.1474 - val_accuracy: 0.9500
Epoch 4/10
2284/2284 [==============================] - 54s 24ms/step - loss: 0.1234 - accuracy: 0.9569 - val_loss: 0.1486 - val_accuracy: 0.9498
Epoch 5/10
2284/2284 [==============================] - 55s 24ms/step - loss: 0.1066 - accuracy: 0.9619 - val_loss: 0.1574 - val_accuracy: 0.9512
Epoch 6/10
2284/2284 [==============================] - 54s 24ms/step - loss: 0.0863 - accuracy: 0.9678 - val_loss: 0.1788 - val_accuracy: 0.9459
Epoch 7/10
2284/2284 [==============================] - 54s 24ms/step - loss: 0.0708 - accuracy: 0.9745 - val_

In [ ]:
indices = find_best(training_details)
print("Best parameter is : ","Neuron :",neurons_num[indices[0]])

#Tune Epoch

In [ ]:
def param_tune(epoch_num,batch_size_num,optimizer_type,dropout_num):
  epochs_num = []
  training_details = []
  global model3
  for i in range(len(epoch_num)):
    # tf.keras.backend.clear_session()
    if i == 0:
      model3 = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedding_dim_pretrained_embedding,weights=[embedding_matrix],input_length=max_length,trainable=False),  
                                    tf.compat.v1.keras.layers.CuDNNLSTM(embedding_dim),
                                    tf.keras.layers.Dropout(dropout_num),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    ])
      model3.compile(loss='binary_crossentropy',optimizer=optimizer_type,metrics=['accuracy'])
      # model3.summary()
      print("Epoch : ",epoch_num[i])
      history = model3.fit(training_padded, train_label, epochs=epoch_num[i], validation_data=(valid_padded, valid_label),batch_size=batch_size_num)
      epochs_num.append(epoch_num[i]) 
      training_details.append({'loss': history.history['loss'][-1],
                              'accuracy':history.history['accuracy'][-1],
                              'val_loss':history.history['val_loss'][-1],
                              'val_accuracy':history.history['val_accuracy'][-1]})
    else:
       print("Epoch : ",epoch_num[i])
       history = model3.fit(training_padded, train_label, epochs=epoch_num[i]-epoch_num[i-1], validation_data=(valid_padded, valid_label),batch_size=batch_size_num)
       epochs_num.append(epoch_num[i]) 
       training_details.append({'loss': history.history['loss'][-1],
                                'accuracy':history.history['accuracy'][-1],
                                'val_loss':history.history['val_loss'][-1],
                                'val_accuracy':history.history['val_accuracy'][-1]})

    # del model3
  return epochs_num,training_details

In [ ]:
e = [1,2,3,4,5]
b_size = 16
opt = 'Adam'
drop = 0.1
epochs_num,training_details = param_tune(e,b_size,opt,drop)
for i in range(len(epochs_num)):
  print("\n","Epoch :",epochs_num[i]," Details : ",training_details[i])

Epoch :  1
2284/2284 [==============================] - 17s 8ms/step - loss: 0.2002 - accuracy: 0.9272 - val_loss: 0.1736 - val_accuracy: 0.9380
Epoch :  2
2284/2284 [==============================] - 12s 5ms/step - loss: 0.1598 - accuracy: 0.9448 - val_loss: 0.1665 - val_accuracy: 0.9404
Epoch :  3
2284/2284 [==============================] - 12s 5ms/step - loss: 0.1511 - accuracy: 0.9478 - val_loss: 0.1616 - val_accuracy: 0.9448
Epoch :  4
2284/2284 [==============================] - 12s 5ms/step - loss: 0.1442 - accuracy: 0.9500 - val_loss: 0.1626 - val_accuracy: 0.9452
Epoch :  5
2284/2284 [==============================] - 12s 5ms/step - loss: 0.1376 - accuracy: 0.9525 - val_loss: 0.1598 - val_accuracy: 0.9470

 Epoch : 1  Details :  {'loss': 0.20022346079349518, 'accuracy': 0.927150309085846, 'val_loss': 0.17355138063430786, 'val_accuracy': 0.9380182027816772}

 Epoch : 2  Details :  {'loss': 0.15978200733661652, 'accuracy': 0.9447743892669678, 'val_loss': 0.16649441421031952, 'v

In [ ]:
indices = find_best(training_details)
print("Best parameter is : ","Epoch :",epochs_num[indices[0]])

Best parameter is :  Epoch : 25


#Tuning part is end

#Test part now

#At first train model using the final tuned parameter and then test with test data

In [18]:
from tensorflow.keras import backend as K

# def confusion_m_ele(y_true, y_pred):
#     true_positive = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     false_negative = possible_positives - true_positive
#     false_positive = predicted_positives - true_positive
#     true_negative = len(test_data) - true_positive - false_positive - false_negative
#     return true_positive,false_positive,false_negative,true_negative

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
    
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# def confusion_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     false_negatives = possible_positives - true_positives
#     false_positives = predicted_positives - true_positives
#     true_negatives = len(y_true) - true_positives - false_positives - false_negatives
#     return np.array([[true_negatives,false_positives],[false_negatives,true_positives]])

#Bi-direction

#Bi-direction with normal Trainable embedding

In [ ]:
model3 = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedding_dim_normal_embedding,input_length=max_length),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.1, recurrent_dropout=0.1)),
                                    tf.keras.layers.Dense(64,activation = 'relu'),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    ])
model3.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc',f1_m,precision_m, recall_m])
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 64)            1920000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,994,369
Trainable params: 1,994,369
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.fit(training_padded, train_label, epochs=4, validation_data=(valid_padded, valid_label),batch_size=32)

Epoch 1/4
1142/1142 [==============================] - 241s 211ms/step - loss: 0.1921 - acc: 0.9362 - f1_m: 0.7842 - precision_m: 0.7904 - recall_m: 0.8025 - val_loss: 0.1604 - val_acc: 0.9480 - val_f1_m: 0.8543 - val_precision_m: 0.8457 - val_recall_m: 0.8858
Epoch 2/4
1142/1142 [==============================] - 240s 210ms/step - loss: 0.1258 - acc: 0.9571 - f1_m: 0.8811 - precision_m: 0.8565 - recall_m: 0.9251 - val_loss: 0.1697 - val_acc: 0.9407 - val_f1_m: 0.8414 - val_precision_m: 0.8081 - val_recall_m: 0.9015
Epoch 3/4
1142/1142 [==============================] - 238s 209ms/step - loss: 0.0943 - acc: 0.9669 - f1_m: 0.9076 - precision_m: 0.8796 - recall_m: 0.9538 - val_loss: 0.2005 - val_acc: 0.9409 - val_f1_m: 0.8392 - val_precision_m: 0.8214 - val_recall_m: 0.8830
Epoch 4/4
1142/1142 [==============================] - 239s 209ms/step - loss: 0.0651 - acc: 0.9751 - f1_m: 0.9293 - precision_m: 0.9104 - recall_m: 0.9611 - val_loss: 0.2543 - val_acc: 0.9307 - val_f1_m: 0.8174 - val

In [ ]:
loss, accuracy, f1_score, precision, recall = model3.evaluate(testing_padded,test_label)
print('Accuracy: %.2f' % (accuracy*100))

381/381 [==============================] - 7s 17ms/step - loss: 0.2349 - acc: 0.9337 - f1_m: 0.8291 - precision_m: 0.7930 - recall_m: 0.8960
Accuracy: 93.37


In [ ]:
# print(loss, accuracy, f1_score, precision, recall)
print('Loss: %.2f' % (loss*100))
print('Accuracy: %.2f' % (accuracy*100))
print('Precision: %.2f' % (precision*100))
print('Recall: %.2f' % (recall*100))
print('F1_score: %.2f' % (f1_score*100))

Loss: 23.49
Accuracy: 93.37
Precision: 79.30
Recall: 89.60
F1_score: 82.91


#Bi-direction with pretrained embedding

In [ ]:
model3 = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedding_dim_pretrained_embedding,weights=[embedding_matrix],input_length=max_length,trainable=False),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.1, recurrent_dropout=0.1)),
                                    tf.keras.layers.Dense(64,activation = 'relu'),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    ])
model3.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc',f1_m,precision_m, recall_m])
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 300)           9000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               186880    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 9,195,201
Trainable params: 195,201
Non-trainable params: 9,000,000
_________________________________________________________________


In [ ]:
model3.fit(training_padded, train_label, epochs=4, validation_data=(valid_padded, valid_label),batch_size=32)

Epoch 1/4
1142/1142 [==============================] - 308s 269ms/step - loss: 0.2113 - acc: 0.9247 - f1_m: 0.7653 - precision_m: 0.7858 - recall_m: 0.7824 - val_loss: 0.1687 - val_acc: 0.9432 - val_f1_m: 0.8396 - val_precision_m: 0.8388 - val_recall_m: 0.8630
Epoch 2/4
1142/1142 [==============================] - 305s 267ms/step - loss: 0.1597 - acc: 0.9473 - f1_m: 0.8544 - precision_m: 0.8464 - recall_m: 0.8854 - val_loss: 0.1591 - val_acc: 0.9468 - val_f1_m: 0.8462 - val_precision_m: 0.8486 - val_recall_m: 0.8665
Epoch 3/4
1142/1142 [==============================] - 304s 266ms/step - loss: 0.1514 - acc: 0.9497 - f1_m: 0.8586 - precision_m: 0.8539 - recall_m: 0.8884 - val_loss: 0.1621 - val_acc: 0.9443 - val_f1_m: 0.8353 - val_precision_m: 0.8595 - val_recall_m: 0.8362
Epoch 4/4
1142/1142 [==============================] - 304s 266ms/step - loss: 0.1466 - acc: 0.9506 - f1_m: 0.8587 - precision_m: 0.8531 - recall_m: 0.8870 - val_loss: 0.1563 - val_acc: 0.9467 - val_f1_m: 0.8488 - val

In [ ]:
loss, accuracy, f1_score, precision, recall = model3.evaluate(testing_padded,test_label)
print('Accuracy: %.2f' % (accuracy*100))

381/381 [==============================] - 8s 21ms/step - loss: 0.1486 - acc: 0.9511 - f1_m: 0.8659 - precision_m: 0.8622 - recall_m: 0.8919
Accuracy: 95.11


In [ ]:
# print(loss, accuracy, f1_score, precision, recall)
print('Loss: %.2f' % (loss*100))
print('Accuracy: %.2f' % (accuracy*100))
print('Precision: %.2f' % (precision*100))
print('Recall: %.2f' % (recall*100))
print('F1_score: %.2f' % (f1_score*100))

Loss: 14.86
Accuracy: 95.11
Precision: 86.22
Recall: 89.19
F1_score: 86.59


#Single Direction

#Single direction with normal Trainable embedding

In [ ]:
model3 = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedding_dim_normal_embedding,input_length=max_length),
                                    tf.keras.layers.LSTM(64, dropout=0.1, recurrent_dropout=0.1),
                                    tf.keras.layers.Dense(64,activation = 'relu'),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    ])
model3.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc',f1_m,precision_m, recall_m])
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 32, 64)            1920000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 1,957,249
Trainable params: 1,957,249
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model3.fit(training_padded, train_label, epochs=4, validation_data=(valid_padded, valid_label),batch_size=32)

Epoch 1/4
1142/1142 [==============================] - 124s 108ms/step - loss: 0.1958 - acc: 0.9345 - f1_m: 0.7656 - precision_m: 0.7710 - recall_m: 0.7870 - val_loss: 0.1588 - val_acc: 0.9476 - val_f1_m: 0.8549 - val_precision_m: 0.8448 - val_recall_m: 0.8892
Epoch 2/4
1142/1142 [==============================] - 124s 109ms/step - loss: 0.1250 - acc: 0.9574 - f1_m: 0.8817 - precision_m: 0.8601 - recall_m: 0.9261 - val_loss: 0.1672 - val_acc: 0.9452 - val_f1_m: 0.8459 - val_precision_m: 0.8434 - val_recall_m: 0.8717
Epoch 3/4
1142/1142 [==============================] - 124s 108ms/step - loss: 0.0890 - acc: 0.9661 - f1_m: 0.9042 - precision_m: 0.8917 - recall_m: 0.9341 - val_loss: 0.1867 - val_acc: 0.9388 - val_f1_m: 0.8301 - val_precision_m: 0.8285 - val_recall_m: 0.8552
Epoch 4/4
1142/1142 [==============================] - 124s 108ms/step - loss: 0.0606 - acc: 0.9757 - f1_m: 0.9287 - precision_m: 0.9304 - recall_m: 0.9405 - val_loss: 0.2306 - val_acc: 0.9316 - val_f1_m: 0.8046 - val

In [ ]:
loss, accuracy, f1_score, precision, recall = model3.evaluate(testing_padded,test_label)
print('Accuracy: %.2f' % (accuracy*100))

381/381 [==============================] - 4s 10ms/step - loss: 0.2111 - acc: 0.9357 - f1_m: 0.8180 - precision_m: 0.8527 - recall_m: 0.8128
Accuracy: 93.57


In [ ]:
# print(loss, accuracy, f1_score, precision, recall)
print('Loss: %.2f' % (loss*100))
print('Accuracy: %.2f' % (accuracy*100))
print('Precision: %.2f' % (precision*100))
print('Recall: %.2f' % (recall*100))
print('F1_score: %.2f' % (f1_score*100))

Loss: 21.11
Accuracy: 93.57
Precision: 85.27
Recall: 81.28
F1_score: 81.80


#Single direction with pretrained embedding

In [19]:
model3 = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,embedding_dim_pretrained_embedding,weights=[embedding_matrix],input_length=max_length,trainable=False),
                                    tf.keras.layers.LSTM(64, dropout=0.1, recurrent_dropout=0.1),
                                    tf.keras.layers.Dense(64,activation = 'relu'),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
                                    ])
model3.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['acc',f1_m,precision_m, recall_m])
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 300)           9000000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                93440     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 9,097,665
Trainable params: 97,665
Non-trainable params: 9,000,000
_________________________________________________________________


In [20]:
model3.fit(training_padded, train_label, epochs=4, validation_data=(valid_padded, valid_label),batch_size=32)

Epoch 1/4
1142/1142 [==============================] - 105s 92ms/step - loss: 0.2103 - acc: 0.9247 - f1_m: 0.7582 - precision_m: 0.7832 - recall_m: 0.7667 - val_loss: 0.1667 - val_acc: 0.9450 - val_f1_m: 0.8465 - val_precision_m: 0.8386 - val_recall_m: 0.8755
Epoch 2/4
1142/1142 [==============================] - 104s 91ms/step - loss: 0.1585 - acc: 0.9474 - f1_m: 0.8487 - precision_m: 0.8468 - recall_m: 0.8766 - val_loss: 0.1580 - val_acc: 0.9471 - val_f1_m: 0.8511 - val_precision_m: 0.8477 - val_recall_m: 0.8759
Epoch 3/4
1142/1142 [==============================] - 104s 91ms/step - loss: 0.1501 - acc: 0.9493 - f1_m: 0.8583 - precision_m: 0.8553 - recall_m: 0.8845 - val_loss: 0.1754 - val_acc: 0.9476 - val_f1_m: 0.8558 - val_precision_m: 0.8409 - val_recall_m: 0.8922
Epoch 4/4
1142/1142 [==============================] - 106s 93ms/step - loss: 0.1462 - acc: 0.9502 - f1_m: 0.8610 - precision_m: 0.8550 - recall_m: 0.8917 - val_loss: 0.1543 - val_acc: 0.9477 - val_f1_m: 0.8547 - val_pre

In [21]:
loss, accuracy, f1_score, precision, recall = model3.evaluate(testing_padded,test_label)
print('Accuracy: %.2f' % (accuracy*100))

381/381 [==============================] - 4s 10ms/step - loss: 0.1467 - acc: 0.9508 - f1_m: 0.8667 - precision_m: 0.8496 - recall_m: 0.9068
Accuracy: 95.08


In [22]:
# print(loss, accuracy, f1_score, precision, recall)
print('Loss: %.2f' % (loss*100))
print('Accuracy: %.2f' % (accuracy*100))
print('Precision: %.2f' % (precision*100))
print('Recall: %.2f' % (recall*100))
print('F1_score: %.2f' % (f1_score*100))

Loss: 14.67
Accuracy: 95.08
Precision: 84.96
Recall: 90.68
F1_score: 86.67
